In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

tokenizer = AutoTokenizer.from_pretrained(
'speakleash/Bielik-7B-v0.1',
token=''
)

model = AutoModelForCausalLM.from_pretrained(
'speakleash/Bielik-7B-v0.1',
quantization_config=BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16),
token='',
)
!nvidia-smi

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 28.00 MiB. GPU 0 has a total capacity of 11.71 GiB of which 28.19 MiB is free. Process 7157 has 10.84 GiB memory in use. Including non-PyTorch memory, this process has 712.00 MiB memory in use. Of the allocated memory 482.53 MiB is allocated by PyTorch, and 7.47 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [3]:
messages = [
    {
        "role": "system",
        "content": "Jesteś asystentem, odpowiadaj na pytania zgodnie ze swoimi przekonaniami politycznymi"
    },
    {
        "role": "user",
        "content": "Sprawozdanie Komisji Finansów Publicznych o rządowym projekcie ustawy o zmianie ustawy o podatku rolnym, ustawy o podatkach i opłatach lokalnych oraz ustawy o opłacie skarbowej."
    },
 
]

print(tokenizer.apply_chat_template(messages, return_tensors="pt"))

ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [21]:
def chat_template(message, history, system_prompt):
    prompt_builder = ["<s>[INST] "]
    if system_prompt:
        prompt_builder.append(f"<<SYS>>\n{system_prompt}\n<</SYS>>\n\n")
    for human, assistant in history:
        prompt_builder.append(f"{human} [/INST] {assistant}</s>[INST] ")
    prompt_builder.append(f"{message} [/INST]")
    return ''.join(prompt_builder)

system_prompt = "Jesteś asystentem, odpowiadaj na pytania zgodnie ze swoimi przekonaniami politycznymi i wyłącznie w języku polskim."
history = [
]
message = "Jakie jest twoje zdanie na temat TVP?"

prompt = chat_template(message, history, system_prompt)
print(prompt)

inputs = tokenizer(prompt, return_tensors="pt")
inputs.to('cuda')

print(inputs)


# model_outputs = model.generate(**inputs,
#                                max_new_tokens=128,
#                                return_dict_in_generate=True,
#                                output_scores=True,
#                                 do_sample=True,
#                                 temperature=0.7)

# generated_tokens_ids = model_outputs.sequences[0]
# answer = tokenizer.decode(generated_tokens_ids)
#print(answer)


<s>[INST] <<SYS>>
Jesteś asystentem, odpowiadaj na pytania zgodnie ze swoimi przekonaniami politycznymi i wyłącznie w języku polskim.
<</SYS>>

Jakie jest twoje zdanie na temat TVP? [/INST]
{'input_ids': tensor([[    1,     1,   733, 16289, 28793,  2087, 18741,  4060,    13, 28798,
          7939, 28884,   390,   827,   308,   366, 28725,  4321, 15625, 28710,
           316,  1150,  1879,   284,  4475,  4877,   686, 21416,  4255,  6797,
         20153, 13840,  7713,   950,   266,   276, 14581,  1160,   472,  3808,
         15416, 28710,   613,  8501, 22660,  3808,  4255,   275,   461, 28864,
          2140,  3907,  1160, 21444, 28723,    13, 28789,   700, 18741,  4060,
            13,    13, 28798,   491,   412, 10572,   989,  2099, 20133,  8768,
          1879,  1637,   270,  5046, 28753, 28804,   733, 28748, 16289, 28793]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1,